# Recorrência de Pontos por Piloto da Temporada de 2025

## 1.0 Ingestão de dados

### 1.1 Visualizando arquivos na Camada Silver

In [0]:
%fs
ls dbfs:/mnt/Formula1/Silver

### 1.2 Carregando os arquivos parquet em DataFrames

In [0]:
dim_driver_details_df = (spark.read.parquet("dbfs:/mnt/Formula1/Silver/Dim_Driver_Details.parquet/"))

fato_race_results_df = (spark.read.parquet("dbfs:/mnt/Formula1/Silver/Fato_Race_Results.parquet/"))

##2.0 Modelagem de Dados

### 2.1 Preparando os dados para análise

In [0]:
# Dos 20 pilotos escalados para a temporada de 2025, 16 possuem histórico de
# corrida, vamos realizar o filtro:

from pyspark.sql.functions import col

dim_pilotos_com_historico = [
    "hamilton", "max_verstappen", "alonso", "leclerc", "sainz",
    "russell", "ocon", "gasly", "norris", "bearman", "stroll",
    "hulkenberg", "lawson", "tsunoda", "piastri", "albon"
]

dim_pilotos_com_historico_df = dim_driver_details_df.filter(
    col("Piloto").isin(dim_pilotos_com_historico)
)

In [0]:
# Junção entre os DataFrames pilotos_temporada_2025_df e race_results_df para análise

join_pilotos_corridas_df = fato_race_results_df.join(
    dim_pilotos_com_historico_df,
    on="Piloto_Id",
    how="right"
)

##3.0 Agregações e métricas

In [0]:
from pyspark.sql import functions as F

# Agrupando por piloto e posição, e contando quantas vezes cada posição ocorreu
recorrencia_pontos_df = join_pilotos_corridas_df.groupBy("Piloto", "Ordem_Posição").agg(
    F.count("Ordem_Posição").alias("Recorrência")
).orderBy("Piloto", "Ordem_Posição")  # Ordenando por Piloto e Posição

##4.0 Tratamento final de dados

### 4.1 Coluna indicando quando os dados foram atualizados

In [0]:
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp

recorrencia_pontos_df = recorrencia_pontos_df.withColumn(
    "Atualizacao",
    date_format(
        from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"),
        "yyyy-MM-dd HH:mm:ss"
    )
)

##5.0 Particionamento e Performance

### 5.1 Salvando arquivo no formato parquet e particionado por Piloto

In [0]:
recorrencia_pontos_df.write \
    .mode("overwrite") \
    .format("parquet") \
    .partitionBy("Piloto") \
    .save("dbfs:/mnt/Formula1/Gold/Recorrência_Pontos_Por_Piloto_Temporada_2025.parquet")

5.2 Visualizando arquivos gravados na Camada Gold

In [0]:
%fs
ls dbfs:/mnt/Formula1/Gold/Recorrência_Pontos_Por_Piloto_Temporada_2025.parquet